# メタデータを登録する
研究データのメタデータを登録するタスクです。

## メタデータ登録フォームを表示する
メタデータを登録するためのフォームを表示します。<br>
登録済みのメタデータが存在する場合は呼び出して表示します。<br>

In [ ]:
# メタデータ登録フォームを表示する
import json
import os
import traceback
from typing import Optional, Union

from IPython.core.display import Javascript
from IPython.display import display, clear_output
import panel as pn
from param.parameterized import Event
from requests.exceptions import RequestException
from dg_mm import MetadataManager

from library.task_director import TaskDirector
from library.utils import dg_web
from library.utils.config import connect as con_config
from library.utils.config import message as msg_config
from library.utils.error import (UnusableVault, ProjectNotExist,
                                 UnauthorizedError, RepoPermissionError)
from library.utils.file import JsonFile
from library.utils.input import get_grdm_connection_parameters
from library.utils.widgets import Button, MessageBox
from library.utils.storage_provider import grdm


pn.extension('floatpanel')
notebook_name = 'metadata.ipynb'


class Metadata(TaskDirector):
    """メタデータを表示し、登録するクラスです。

    Attributes:
        instance:
            form_box (pn.WidgetBox): フォーム表示用のウィジット
            metadata_manager_form (pn.WidgetBox): メタデータマネージャー用のウィジット
            schema_form (dg_web.Form): DG-Web入力フォームの操作クラス
            runcrate_form (dg_web.RunCrateForm): RunCrate入力フォームの操作クラス
            message_box (pn.WidgetBox): メッセージ表示用のウィジット
            msg_output (MessageBox): このクラス内で発生するメッセージを表示するためのクラス
            token (str): パーソナルアクセストークン
            project_id (str): プロジェクトID
            remote_path (str): リモート上のメタデータのパス
            submit_button_title (str): 保存ボタンのタイトル
            submit_button (Button): 保存ボタン
            dg_web_url (str): DG-webのURL
            grdm_url (str): GRDMのURL
            grdm (Grdm): GRDMへのアクセスを行うクラス
    """

    def __init__(self, working_path: str) -> None:
        """Metadata コンストラクタのメソッドです。

        Args:
            working_path(str):実行Notebookファイルパス
        """
        super().__init__(working_path, notebook_name)

        self.metadata_manager_form = self.create_metadata_manager_form()
        self.schema_form = dg_web.Form()
        self.runcrate_form = dg_web.RunCrateForm()

        self.jsonfile_path = os.path.join(
            self._abs_root_path, 'data_governance/library/data/metadata_order.json'
        )

        # フォームボックス
        self.form_box = pn.WidgetBox()
        self.form_box.width = 900
        self.form_box.append(self.metadata_manager_form)
        self.metadata_manager_form.width = 900
        self.form_box.append(self.schema_form.form_box)
        self.schema_form.form_box.width = 900
        self.form_box.append(self.runcrate_form.form_box)
        self.runcrate_form.form_box.width = 900

        # メッセージ用ボックス
        self.message_box = pn.WidgetBox()
        self.msg_output = MessageBox()
        self.msg_output.width = 900
        self.message_box.append(self.msg_output)
        self.schema_form.msg_output.width = 900
        self.message_box.append(self.schema_form.msg_output)
        self.runcrate_form.msg_output.width = 900
        self.message_box.append(self.runcrate_form.msg_output)

        self.dg_web_url = con_config.get('DG_WEB', 'BASE_URL')
        self.grdm_url = con_config.get('GRDM', 'BASE_URL')
        self.grdm = grdm.Grdm()

    def get_grdm_params(self) -> tuple[str, str]:
        """GRDMのトークンとプロジェクトIDを取得するメソッドです。

        Returns:
            str:GRDMのトークンの値を返す。
            str:プロジェクトIDの値を返す。
        """

        token = ""
        project_id = ""
        try:
            token, project_id = get_grdm_connection_parameters(self.grdm_url)
        except UnusableVault as e:
            message = msg_config.get('form', 'no_vault')
            self.msg_output.update_error(message)
            self.log.error(f'{message}\n{traceback.format_exc()}')
        except RepoPermissionError:
            message = msg_config.get('form', 'insufficient_permission')
            self.msg_output.update_error(message)
            self.log.error(f'{message}\n{traceback.format_exc()}')
        except ProjectNotExist as e:
            self.msg_output.update_error(str(e))
            self.log.error(traceback.format_exc())
        except RequestException as e:
            message = msg_config.get('DEFAULT', 'connection_error')
            self.msg_output.update_error(f'{message}\n{str(e)}')
            self.log.error(f'{message}\n{traceback.format_exc()}')
        except Exception:
            message = f'## [INTERNAL ERROR] : {traceback.format_exc()}'
            self.msg_output.update_error(message)
            self.log.error(message)
        return token, project_id

    @TaskDirector.task_cell("1")
    def generate_form_section(self):
        """取得したデータを表示するメソッドです。"""
        # タスク開始によるサブフローステータス管理JSONの更新
        self.doing_task()

        try:
            # パラメータの取得
            self.token, self.project_id = self.get_grdm_params()
            self.remote_path = con_config.get('DG_WEB', 'METADATA_PATH')
            self.schema = None
            clear_output()

            # データの取得
            if self.token and not self.msg_output.has_message():
                self.schema = self.get_schema()
                data = self.get_data(
                    token=self.token,
                    project_id=self.project_id, remote_path=self.remote_path
                )
                runcrates = self.get_runncrate(
                    token=self.token, project_id=self.project_id
                )

                sorted_schema = self.schema_form.sort_order(self.schema, self.jsonfile_path)

            # フォーム定義
            if self.schema and not self.msg_output.has_message():
                schema = self.runcrate_form.pop_schema(sorted_schema)
                self.schema_form.create_widgets(schema, data)
                self.runcrate_form.create_widget(runcrates, data)
                self.submit_button_title = msg_config.get('save', 'submit')
                self.submit_button = Button(width=500)
                self.submit_button.set_looks_init(self.submit_button_title)
                self.submit_button.on_click(self.submit)
                self.form_box.append(self.submit_button)
        except Exception:
            message = f'## [INTERNAL ERROR] : {traceback.format_exc()}'
            self.msg_output.update_error(message)
            self.log.error(message)

        # フォーム表示
        pn.extension()
        form_section = pn.WidgetBox()
        form_section.append(self.form_box)
        form_section.append(self.msg_output)
        display(form_section)
        display(Javascript('IPython.notebook.save_checkpoint();'))

    def get_schema(self) -> dict:
        """jsonschemaを取得するメソッドです。

        Returns:
            dict:jsonschemaの値を返す。
        """
        schema = {}
        try:
            dgwebapi = dg_web.Api()
            schema = dgwebapi.get_metadata_schema(self.dg_web_url)
        except RequestException as e:
            message = msg_config.get('DEFAULT', 'connection_error')
            self.msg_output.update_error(f'{message}\n{str(e)}')
            self.log.error(f'{message}\n{traceback.format_exc()}')
        return schema

    def get_data(self, token: str, project_id: str, remote_path: str) -> Optional[dict]:
        """指定したファイルの内容を取得するメソッドです。

        Args:
            token(str):パーソナルアクセストークン
            project_id(str):プロジェクトID
            remote_path(str:リモート先のパス

        Returns:
            Optional[dict]:ファイルの内容を返す。
        """
        data = None
        try:
            data = self.grdm.download_json_file(
                token=token, base_url=self.grdm_url,
                project_id=project_id, remote_path=remote_path
            )
        except FileNotFoundError:
            data = None
        except json.JSONDecodeError:
            data = {}
        except UnauthorizedError:
            message = msg_config.get('form', 'token_unauthorized')
            self.msg_output.update_warning(message)
            self.log.warning(f'{message}\n{traceback.format_exc()}')
        except RequestException as e:
            message = msg_config.get('dg_web', 'get_data_error')
            self.msg_output.update_error(f'{message}\n{str(e)}')
            self.log.error(f'{message}\n{traceback.format_exc()}')
        except Exception as e:
            message = msg_config.get('dg_web', 'get_data_error')
            self.msg_output.update_error(message)
            self.log.error(f'{message}\n{traceback.format_exc()}')
        return data

    def get_runncrate(self, token: str, project_id: str) -> list:
        """Governed Runのindex.jsonの内容を取得するメソッドです。

        Args:
            token(str):パーソナルアクセストークン
            project_id(str):プロジェクトID

        Returns:
            list:index.jsonの内容を返す。
        """
        crates = []
        try:
            crates = self.grdm.download_json_file(token, self.grdm_url, project_id, con_config.get('DG_WEB', 'GOVRUN_INDEX_PATH'))
        except (FileNotFoundError, json.JSONDecodeError):
            crates = []
        except UnauthorizedError:
            message = msg_config.get('form', 'token_unauthorized')
            self.msg_output.update_warning(message)
            self.log.warning(f'{message}\n{traceback.format_exc()}')
        except RequestException as e:
            message = msg_config.get('dg_web', 'get_data_error')
            self.msg_output.update_error(f'{message}\n{str(e)}')
            self.log.error(f'{message}\n{traceback.format_exc()}')
        except Exception as e:
            message = msg_config.get('dg_web', 'get_data_error')
            self.msg_output.update_error(message)
            self.log.error(f'{message}\n{traceback.format_exc()}')
        return crates

    @TaskDirector.callback_form('メタデータを保存する')
    def submit(self, event):
        """メタデータを保存するメソッドです。"""
        file_path = os.path.join(self._abs_root_path, self.remote_path)
        tmpfile = JsonFile(file_path)
        try:
            data = self.schema_form.get_data()
            runcrate_data = self.runcrate_form.get_data()
            data.update(runcrate_data)
            tmpfile.write(data)
            self.submit_button.disabled = True
            self.submit_button.set_looks_processing(msg_config.get('save', 'doing'))
            self.grdm.sync(
                token=self.token,
                base_url=self.grdm_url,
                project_id=self.project_id,
                abs_source=file_path,
                abs_root=self._abs_root_path
            )
        except UnauthorizedError:
            message = msg_config.get('form', 'token_unauthorized')
            self.msg_output.update_warning(message)
            self.log.warning(f'{message}\n{traceback.format_exc()}')
            return
        except RequestException as e:
            message = msg_config.get('DEFAULT', 'connection_error')
            self.msg_output.update_error(f'{message}\n{str(e)}')
            self.log.error(f'{message}\n{traceback.format_exc()}')
            return
        except Exception:
            message = f'## [INTERNAL ERROR] : {traceback.format_exc()}'
            self.msg_output.update_error(message)
            self.log.error(message)
            return
        finally:
            tmpfile.remove(missing_ok=True)
            self.submit_button.disabled = False
            self.submit_button.set_looks_init(self.submit_button_title)

        self.form_box.clear()
        self.msg_output.update_success(msg_config.get('dg_web', 'saved_metadata'))
        # タスク実行の完了情報を該当サブフローステータス管理JSONに書き込む
        self.done_task()

    def create_metadata_manager_form(self):
        """メタデータマネージャー用のフォームを作成するメソッドです。"""
        form_box = pn.WidgetBox()
        # メタデータ取得ボタン
        get_metadata_button = Button(width=150)
        get_metadata_button.set_looks_init(msg_config.get('metadata_manager', 'get_metadata'))
        # 未入力のみ取得ボタン
        get_blank_metadata_button = Button(width=150)
        get_blank_metadata_button.set_looks_init(msg_config.get('metadata_manager', 'get_blank_metadata'))
        # 確認メッセージ
        confirm_message = self.create_metadata_manager_confirm_message()

        def on_click_get_metadata(event: Event):
            """メタデータ取得ボタン押下時のコールバックメソッドです。"""
            # ボタンを処理中に変更
            event.obj.set_looks_processing()
            # 確認メッセージの表示
            confirm_message.visible = True

        def on_click_get_blank_metadata(event: Event):
            """未入力のみ取得ボタン押下時のコールバックメソッドです。"""
            # ボタンを処理中に変更
            event.obj.set_looks_processing()

            try:
                # ストレージ(GRDM)からメタデータ取得
                mm = MetadataManager()
                metadata_from_mm = mm.get_metadata('RF', 'GRDM', self.token, self.project_id)

                # フォームにマージする(値が存在する場合は変更しない)
                self.merge_metadata_form(metadata_from_mm, overwrite=False)
            except Exception:
                message = msg_config.get('metadata_manager', 'error_message_get_metadata')
                self.msg_output.update_error(message)
                self.log.error(f'{message}\n{traceback.format_exc()}')

            # ボタンをもとに戻す
            event.obj.set_looks_init(msg_config.get('metadata_manager', 'get_blank_metadata'))

        # それぞれのボタンにイベントハンドラを設定
        get_metadata_button.on_click(on_click_get_metadata)
        get_blank_metadata_button.on_click(on_click_get_blank_metadata)

        form_box.append(pn.Row(get_metadata_button, get_blank_metadata_button))
        form_box.append(pn.Row(confirm_message))

        return form_box

    def create_metadata_manager_confirm_message(self):
        """メタデータマネージャー用の確認メッセージを作成するメソッドです。"""
        ok_button = Button(width=75)
        ok_button.set_looks_init(msg_config.get('metadata_manager', 'yes'))
        ng_button = Button(width=75)
        ng_button.set_looks_init(msg_config.get('metadata_manager', 'no'))
        confirm_message = pn.layout.FloatPanel(
            pn.Row(msg_config.get('metadata_manager', 'confirm_message')),
            pn.Row(
                pn.Spacer(width=25),
                ok_button,
                pn.Spacer(width=75),
                ng_button
            ),
            width=350,
            height=130,
            config={'header': None},
            visible=False,
        )

        def on_click_ok(event: Event):
            """はい押下時のコールバックメソッドです。"""
            # メッセージの非表示
            confirm_message.visible = False

            try:
                # ストレージ(GRDM)からメタデータ取得
                mm = MetadataManager()
                metadata_from_mm = mm.get_metadata('RF', 'GRDM', self.token, self.project_id)

                # フォームにマージする
                self.merge_metadata_form(metadata_from_mm, overwrite=True)
            except Exception:
                message = msg_config.get('metadata_manager', 'error_message_get_metadata')
                self.msg_output.update_error(message)
                self.log.error(f'{message}\n{traceback.format_exc()}')

            # メタデータ取得ボタンをもとに戻す
            get_metadata_button = self.metadata_manager_form[0][0]
            get_metadata_button.set_looks_init(msg_config.get('metadata_manager', 'get_metadata'))

        def on_click_ng(event: Event):
            """いいえ押下時のコールバックメソッドです。"""
            # メッセージの非表示
            confirm_message.visible = False
            # メタデータ取得ボタンをもとに戻す
            get_metadata_button = self.metadata_manager_form[0][0]
            get_metadata_button.set_looks_init(msg_config.get('metadata_manager', 'get_metadata'))

        # それぞれのボタンにイベントハンドラを設定
        ok_button.on_click(on_click_ok)
        ng_button.on_click(on_click_ng)

        return confirm_message

    def merge_metadata_form(self, metadata: dict, overwrite: bool):
        """指定したメタデータを入力中のメタデータにマージします。

        Args:
            metadata (dict): マージするメタデータ(マージ元)
            overwrite (bool): 上書きフラグ。Trueの場合はフォームの値を上書きする。
        """

        # 入力中のメタデータ
        input_metadata = self.schema_form.get_data()

        # メタデータのマージ
        self.merge_metadata(metadata, input_metadata, overwrite)

        # マージしたメタデータを画面に表示
        sorted_schema = self.schema_form.sort_order(self.schema, self.jsonfile_path)
        schema = self.runcrate_form.pop_schema(sorted_schema)
        self.schema_form.create_widgets(schema, input_metadata)

    def merge_metadata(self, src: Union[dict, list], dst: Union[dict, list], overwrite: bool):
        """メタデータのマージ

        Args:
            src (Union[dict, list]): マージ元のメタデータ
            dst (Union[dict, list]): マージ先のメタデータ
            overwrite (bool): 上書きフラグ Trueの場合はマージ先の値をマージ元の値で上書きする
        """
        if isinstance(src, list):
            # 空のリストの場合
            if not src:
                # 何もしない
                pass
            # オブジェクトのリストの場合
            elif isinstance(src[0], dict):
                for index, item in enumerate(src):
                    if len(dst) <= index:
                        dst.append({})
                    self.merge_metadata(item, dst[index], overwrite)
            # 値のリストの場合
            else:
                if not dst:
                    # 項目が存在しない場合は値を設定
                    dst.extend(src)
                elif overwrite:
                    # 上書きの場合は値を設定
                    dst.clear()
                    dst.extend(src)
        elif isinstance(src, dict):
            for key, value in src.items():
                if isinstance(value, list):
                    if key not in dst:
                        dst[key] = []
                    self.merge_metadata(value, dst[key], overwrite)
                else:
                    if not value:
                        continue
                    if key not in dst:
                        # 項目が存在しない場合は値を設定
                        dst[key] = value
                    elif overwrite:
                        # 上書きの場合は値を設定
                        dst[key] = value


Metadata(os.path.abspath('__file__')).generate_form_section()

## サブフローメニューを表示する

サブフローメニューへ遷移するボタンを表示します。

In [ ]:
# サブフローメニューを表示する
import os
from library.task_director import TaskDirector

notebook_name = 'metadata.ipynb'
TaskDirector(os.path.abspath('__file__'), notebook_name).return_subflow_menu()